High Cost

In [0]:
# Install required Excel reader
%pip install openpyxl

# Import libraries
from boxsdk import JWTAuth, Client
from io import BytesIO
import pandas as pd
import re
from pyspark.sql.functions import col, trim, split, size, when

# Authenticate with Box using secrets from Databricks
auth = JWTAuth(
    client_id=dbutils.secrets.get(scope="game_boe_secrets", key="box_client_id"),
    client_secret=dbutils.secrets.get(scope="game_boe_secrets", key="box_client_secret"),
    enterprise_id=dbutils.secrets.get(scope="game_boe_secrets", key="box_enterprise_id"),
    jwt_key_id=dbutils.secrets.get(scope="game_boe_secrets", key="box_jwt_key_id"),
    rsa_private_key_data=dbutils.secrets.get(scope="game_boe_secrets", key="rsa_private_key_data"),
    rsa_private_key_passphrase=dbutils.secrets.get(scope="game_boe_secrets", key="box_private_key_pass_phrase")
)

client = Client(auth)

# Download Excel file from Box
file_id = 'your_file_id'
file_stream = BytesIO(client.file(file_id).content())
df = pd.read_excel(file_stream)

# ✅ Fix Governance Date parsing
if 'Governance Date' in df.columns:
    df['Governance Date'] = pd.to_datetime(df['Governance Date'], errors='coerce')

# Clean column names
df.columns = [re.sub(r'\W+', '_', col).strip('_') for col in df.columns]
df.replace("NULL", pd.NA, inplace=True)

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Basic cleanup
spark_df = spark_df.withColumn("Total_Contract_Cost", col("Total_Contract_Cost").cast("double"))
for column in spark_df.columns:
    if dict(spark_df.dtypes)[column] == 'string':
        spark_df = spark_df.withColumn(column, trim(col(column)))
if "Contract_Lead" in spark_df.columns:
    spark_df = spark_df.withColumn("Contract_Lead", split(col("Contract_Lead"), ","))

# Extract Vendor and Description from Contract_Name
if "Contract_Name" in spark_df.columns:
    parts = split(col("Contract_Name"), "\|")
    spark_df = spark_df \
        .withColumn("Vendor", when(size(parts) >= 1, trim(parts.getItem(0)))) \
        .withColumn("Description", when(size(parts) >= 2, trim(parts.getItem(1))))

# Save to Delta
spark_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("development.team_game_boe.game_high_cost_active")

# ✅ Display contents of the table
spark.sql("""
    SELECT Vendor, Description, Nike_Contract_ID, Governance_Required, Governance_Date, Governance, Comments
    FROM development.team_game_boe.game_high_cost_active
""").show(truncate=False)


1 Milion +

In [0]:
from boxsdk import JWTAuth, Client
from io import BytesIO
import pandas as pd
import re
from pyspark.sql.functions import col, trim, split, size, when

# Authenticate and download Excel file from Box
auth = JWTAuth(
    client_id=dbutils.secrets.get(scope="game_boe_secrets", key="box_client_id"),
    client_secret=dbutils.secrets.get(scope="game_boe_secrets", key="box_client_secret"),
    enterprise_id=dbutils.secrets.get(scope="game_boe_secrets", key="box_enterprise_id"),
    jwt_key_id=dbutils.secrets.get(scope="game_boe_secrets", key="box_jwt_key_id"),
    rsa_private_key_data=dbutils.secrets.get(scope="game_boe_secrets", key="rsa_private_key_data"),
    rsa_private_key_passphrase=dbutils.secrets.get(scope="game_boe_secrets", key="box_private_key_pass_phrase")
)
client = Client(auth)
file_stream = BytesIO(client.file('your_file_id').content())
df = pd.read_excel(file_stream)

# ✅ Fix Governance Date parsing
if 'Governance Date' in df.columns:
    df['Governance Date'] = pd.to_datetime(df['Governance Date'], errors='coerce')

# Clean column names
df.columns = [re.sub(r'\W+', '_', col).strip('_') for col in df.columns]
df.replace("NULL", pd.NA, inplace=True)

# Ensure all columns are string type before creating Spark DataFrame
df = df.astype(str)

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Basic cleanup
spark_df = spark_df.withColumn("Total_Contract_Cost", col("Total_Contract_Cost").cast("double"))
for column in spark_df.columns:
    if dict(spark_df.dtypes)[column] == 'string':
        spark_df = spark_df.withColumn(column, trim(col(column)))
if "Contract_Lead" in spark_df.columns:
    spark_df = spark_df.withColumn("Contract_Lead", split(col("Contract_Lead"), ","))

# Extract Vendor and Description from Contract_Name
if "Contract_Name" in spark_df.columns:
    parts = split(col("Contract_Name"), "\|")
    spark_df = spark_df \
        .withColumn("Vendor", when(size(parts) >= 1, trim(parts.getItem(0)))) \
        .withColumn("Description", when(size(parts) >= 2, trim(parts.getItem(1))))

# Save to Delta
spark_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("development.team_game_boe.game_1m_cost_active")

# ✅ Display contents of the table
spark.sql("""
    SELECT Vendor, Description, Nike_Contract_ID, Governance_Required, Governance_Date, Governance, Comments
    FROM development.team_game_boe.game_1m_cost_active
""").show(truncate=False)


Airtable

In [0]:
# Databricks notebook source
# ===== Install dependencies =====
%pip install --upgrade typing_extensions pyairtable
%pip install xlrd==2.0.1 openpyxl

# ===== Imports =====
import pandas as pd
from pyspark.sql.functions import current_timestamp, expr
import requests
import datetime as dt

# ===== Secrets =====
airtable_access_token = dbutils.secrets.get(scope="game_boe_secrets", key="airtable_access_token")

# ===== Airtable Config =====
base_id = 'your_base_id'
contracts_table_id = 'your_table_id'

# ===== Functions =====
def fetch_airtable_data(api_key, base_id, table_id):
    """Fetch all rows from Airtable table into pandas DataFrame."""
    url = f"https://api.airtable.com/v0/{base_id}/{table_id}"
    headers = {"Authorization": f"Bearer {api_key}"}
    all_records, offset = [], None
    while True:
        params = {}
        if offset:
            params['offset'] = offset
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            raise Exception(f"Error fetching data from Airtable: {response.text}")
        data = response.json()
        records = data.get('records', [])
        all_records.extend([r.get('fields', {}) for r in records])
        offset = data.get('offset')
        if not offset:
            break
    return pd.DataFrame(all_records)

def prepare_data(df, new_columns, date_columns):
    """Rename columns and format date columns."""
    df = df.copy()
    df.rename(columns=new_columns, inplace=True)
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')\
                         .fillna(pd.Timestamp('1900-01-01'))\
                         .dt.strftime('%Y-%m-%d')
    return df

def standardize_cols(df):
    """Make column names Spark-safe."""
    cols = (pd.Series(df.columns)
              .str.replace(' ', '_', regex=False)
              .str.replace('(', '', regex=False)
              .str.replace(')', '', regex=False)
              .str.replace('/', '_', regex=False)
              .str.replace('-', '_', regex=False)
              .str.replace(r'[^A-Za-z0-9_]', '', regex=True))
    df.columns = cols
    return df

# ===== Pull Airtable Data =====
contracts_df = fetch_airtable_data(airtable_access_token, base_id, contracts_table_id)

# ===== Prepare Data =====
# Customize these mappings based on your Airtable schema
contracts_df = prepare_data(contracts_df,
    {'Example Column': 'Renamed Column'},  # Replace with actual mappings
    ['Date Column 1', 'Date Column 2']     # Replace with actual date columns
)

contracts_df = standardize_cols(contracts_df)

# ===== Convert to Spark & Write Delta =====
contracts_sdf = spark.createDataFrame(contracts_df)

spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

contracts_sdf = contracts_sdf.withColumn("upload_timestamp", current_timestamp()) \
                             .withColumn("upload_timestamp", expr("from_utc_timestamp(upload_timestamp, 'America/Los_Angeles')"))

contracts_sdf.write.mode("overwrite").format("delta").saveAsTable("development.team_game_boe.airtable_contracts_bronze")

display(contracts_sdf)
